# Análisis de valores faltantes

In [2]:
import matplotlib.pyplot as plt 
import missingno
import numpy as np
import pandas as pd
import pyreadr
import seaborn as sns
import upsetplot

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
sys.path.append("..")

import modules.utils.paths as path
import modules.utils.get_data as get_data

In [5]:
%run pandas-missing-extension.ipynb

**Carga de datos**

In [6]:
datasets_names = ("oceanbuoys", "pedestrian", "riskfactors")
datasets_df = get_data.from_r(datasets_names, ".rda")
print(datasets_df.keys())

locals().update(**datasets_df)
del datasets_df

dict_keys(['oceanbuoys_df', 'pedestrian_df', 'riskfactors_df'])


## Tabulación de valores faltantes

In [7]:
riskfactors_df.isna()

,state,sex,age,weight_lbs,height_inch,bmi,marital,pregnant,children,education,...,smoke_100,smoke_days,smoke_stop,smoke_last,diet_fruit,diet_salad,diet_potato,diet_carrot,diet_vegetable,diet_juice
0,False,False,False,False,False,False,False,True,False,False,...,False,True,True,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False,True,False,False,...,False,True,True,True,False,False,False,False,False,False
2,False,False,False,False,False,False,False,True,False,False,...,False,True,True,True,False,False,False,False,False,False
3,False,False,False,False,False,False,False,True,False,False,...,False,True,True,True,True,True,True,True,True,True
4,False,False,False,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,False,False,False,False,False,False,False,True,False,False,...,False,True,True,True,False,False,False,False,False,False
241,False,False,False,False,False,False,False,True,False,False,...,False,True,True,True,False,False,False,False,False,False
242,False,False,False,False,False,False,False,True,False,False,...,False,False,True,False,False,False,False,False,False,False
243,False,False,False,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False


### Resumén básico de valores faltantes

- Cuántos valores deberían existir
- Cuántos valores faltan
- Cuántos valores realmente existen
- Cuántas observaciones y variables existen

In [8]:
# Total de elementos del dataframe
print("Total de elementos: ", riskfactors_df.size)

# Total de valores completos
print("Completos: ", riskfactors_df.missing.number_complete())

# Total de valores faltantes
print("Faltantes: ", riskfactors_df.missing.number_missing())

# Observaciones y variables
print(
  f"Observaciones: {riskfactors_df.shape[0]}",
  f"Variables: {riskfactors_df.shape[1]}",
  sep="\n"
)

Total de elementos:  8330
Completos:  7144
Faltantes:  1186
Observaciones: 245
Variables: 34


### Resúmenes tabulares de valores faltantes

#### Variables / Columnas

**Por variable:**
La función personalizada `missing_variable_summary` devuelve `DataFrame` con un resumen por cada viariable en el dataset.

Columnas:
- `variable`: Nombre de la variable (columna del dataset)
- `n_missing`: Número de valores faltantes para la variable.
- `n_cases`: Número total de valores que deberían existir para la variable.
- `pct_missing`: Porcentaje de valores faltantes respecto al total de valores; `n_missing / n_cases`

In [9]:
riskfactors_df.missing.missing_variable_summary().sort_values(by="pct_missing", ascending=False)

,variable,n_missing,n_cases,pct_missing
7,pregnant,215,245,87.755102
26,smoke_stop,212,245,86.530612
27,smoke_last,161,245,65.714286
23,drink_average,135,245,55.102041
22,drink_days,134,245,54.693878
25,smoke_days,128,245,52.244898
17,health_poor,113,245,46.122449
5,bmi,11,245,4.489796
3,weight_lbs,10,245,4.081633
32,diet_vegetable,8,245,3.265306


El resumen anterior podemos observar lo siguiente:

- 7 variables tienen  `> 45%` de datos faltantes.
- Las variables `pregnant` y `smoke_stop` poseen `> 85%` de valores faltantes (¿valdra la pena dejar estas variables en el DataFrame para un posterior análisis?).
- Solo 10 variables cuentaon el $100\%$ de los valores.
- Al resto de las variables con datos faltantes no pasan de un $5\%$.

In [10]:
riskfactors_df.missing.missing_variable_table()

,n_missing_in_variable,n_variables,pct_variables
0,0,10,29.411765
1,8,6,17.647059
2,2,4,11.764706
3,3,3,8.823529
4,1,2,5.882353
5,10,1,2.941176
6,11,1,2.941176
7,113,1,2.941176
8,128,1,2.941176
9,134,1,2.941176


La tabla obtenida con la función `missing_variable_table` nos dice a cuantas variables tienen la misma cantidad de datos faltantes.

Columnas:
- `n_missing_in_variable`: Cantidad de datos faltantes.
- `n_variables`: Número de variables que poseen `n_missing_in_variable` datos faltantes.
- `pct_variables`: Porcentaje correspondiente al número de variables respecto al total de las mismas en el dataset.

#### Casos / Observaciones / Filas

**Por observaciones:**
La función personalizada `missing_case_summary` devuelve `DataFrame` con un resumen por cada observación/fila en el dataset.

Columnas:
- `case`: Index (número de vilas) de la observación en el `DataFrame` original.
- `n_missing`: Número total de columnas/variables en las cuáles no hay valor (`nan`, `None`, `NA`).
- `pct_missing`: Porcentaje de valores faltantes respecto al total de valores; `n_missing / número de columnas`

In [11]:
missing_case_sumary = riskfactors_df.missing.missing_case_summary().sort_values(by="pct_missing", ascending=False)
missing_case_sumary

,case,n_missing,pct_missing
90,90,15,41.666667
132,132,15,41.666667
3,3,12,33.333333
184,184,12,33.333333
48,48,12,33.333333
...,...,...,...
15,15,2,5.555556
215,215,1,2.777778
185,185,1,2.777778
199,199,1,2.777778


En el resumen anterior se puede resaltar lo siguiente:
- Cada fila arroja el número de total decolumnas que no tienen datos.
- Las filas 90 y 132 son las observaciones con mayores valores faltantes ($41.66\%$).
- A Las 245 tienen almenos un valor faltante.

¿Cuántos casos tienen la misma cantidad de columnas son datos?

In [12]:
(
  missing_case_sumary
  .value_counts("n_missing", sort=True)
  .reset_index(name="count")
)

,n_missing,count
0,4,49
1,5,45
2,7,39
3,6,36
4,2,31
5,3,30
6,1,4
7,8,3
8,12,3
9,15,2


### Intervalos de valores faltantes

Mediante un resumen por bloques, vamos contando cuantos valores hacen faltan en intervalos. Con la función personalizada  `missing_variable_span` logramos hacer el contedo por variable.

Columnas:
- `span_counter`: Número de intervalo.
- `n_missing`: Total de valores valtantes en el intervalo.
- `n_complete`: Total de valores completos en el intervalo. 
- `pct_missing`: Porcentaje que representa `n_missing` en el intervalo. O bien porcentaje de faltnaes.
- `pct_complete`:Porcentaje que representa `n_complete` en el intervalo. O bien porcentaje de completos.

In [13]:
(
  riskfactors_df
  .missing
  .missing_variable_span(variable="weight_lbs", span_every=30)
)

,span_counter,n_missing,n_complete,pct_missing,pct_complete
0,0,1,29,3.333333,96.666667
1,1,0,30,0.000000,100.000000
2,2,3,27,10.000000,90.000000
3,3,2,28,6.666667,93.333333
4,4,1,29,3.333333,96.666667
5,5,0,30,0.000000,100.000000
6,6,2,28,6.666667,93.333333
7,7,1,29,3.333333,96.666667
8,8,0,5,0.000000,100.000000


En intervalores de $30$ observaciones para la variable `weight_lbs` podemos observar:
- Son pocos los datos faltantes en la variable.
- Al menos 3 de los intervalos cuentan con el $100\%$ de dados completos.
- El intervalo #3 es el que cuenta con más datos faltantes, solo un $10\%$ 

### _Run length_ de valores faltantes

_Run length_: Rachas de valores faltantes, va va contando los valores completos y cuando encuentra valores faltnates rompe la racha y empieza a contar los faltnates hasta que encuentre valores completos de nuevo (ciclo que se rompe hasta llegar al final de las observaciones). La función personalizada missing_variable_run nos da un dataframe con dicha información.

Columnas:
- `run_length`: Número de observaciones.
- `is_na`: Indica si las observaciones en `run_length` estan completas o son valores faltnates.
  - `complete`: Observaciones con los datos completos.
  - `missing`: Observaciones que no tienen valores.

In [16]:
(
  riskfactors_df
  .missing
  .missing_variable_run(variable="weight_lbs")
)

,run_length,is_na
0,14,complete
1,1,missing
2,45,complete
3,1,missing
4,5,complete
5,1,missing
6,12,complete
7,1,missing
8,10,complete
9,2,missing
